In [43]:
import zipfile
with zipfile.ZipFile('module-5-assignment-1-validation-idx.json.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [123]:
import pandas as pd
import numpy as np
loans=pd.read_csv('lending-club-data.csv')
loans

/home/sikander/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.143500,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.393200,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.259550,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.275850,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.215330,20141201T000000,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122602,9856168,11708132,6000,6000,6000,60 months,23.40,170.53,E,E5,...,1.0,0.0,1.0,1.0,1,4.487630,20190101T000000,0,1,0
122603,9795013,11647121,15250,15250,15250,36 months,17.57,548.05,D,D2,...,0.4,0.0,0.0,1.0,0,10.117800,20170101T000000,0,0,0
122604,9695736,11547808,8525,8525,8525,60 months,18.25,217.65,D,D3,...,0.6,0.0,1.0,1.0,0,6.958120,20190101T000000,0,1,0
122605,9684700,11536848,22000,22000,22000,60 months,19.97,582.50,D,D5,...,1.0,1.0,0.0,1.0,0,8.961540,20190101T000000,1,0,1


In [124]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
# loans = loans.remove_column('bad_loans')

In [125]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loans'  
loans = loans[features + [target]]

In [126]:
import json
with open('module-5-assignment-1-train-idx.json') as imp:
    train_idx = json.load(imp)
with open('module-5-assignment-1-validation-idx.json') as imp:
    validation_idx = json.load(imp)    

In [134]:
categorical_feat=[]
for feat_name, feat_type in zip(loans.columns, loans.dtypes):
    if feat_type == object:
        categorical_feat.append(feat_name)

In [133]:
from sklearn.preprocessing import OneHotEncoder
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
for i in categorical_feat:
    enc_df = pd.DataFrame(enc.fit_transform(loans[[i]]).toarray())
    
    loans = loans.drop(i, axis=1)
#     print(enc_df)
    for j in enc_df.columns:
        loans[j]=enc_df[j]
loans.head(2)          

,short_emp,emp_length_num,dti,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,safe_loans,0,1,...,25,26,27,28,29,30,31,32,33,34
0,0,11,27.65,1,1,83.7,0.0,1,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,1.00,1,1,9.4,0.0,-1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
train=loans.iloc[train_idx]
valid=loans.iloc[validation_idx]

In [77]:
from sklearn import tree

model1 = tree.DecisionTreeClassifier(max_depth=6)
model2=tree.DecisionTreeClassifier(max_depth=2)

In [79]:
# train[target]
x=train.drop(target,axis=1)
y=train[target].to_numpy()
model1.fit(x,y)
x_valid=valid.drop(target,axis=1)
y_valid=valid[target].to_numpy

In [81]:
y1=model1.predict(x_valid)

In [83]:
model2.fit(x,y)
y2=model2.predict(x_valid)

In [84]:
validation_safe_loans = valid[valid[target] == 1]
validation_risky_loans = valid[valid[target] == -1]

sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
sample_validation_data

,short_emp,emp_length_num,dti,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,safe_loans,0,1,...,25,26,27,28,29,30,31,32,33,34
19,0,11,11.18,1,1,82.4,0.0,1,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0,10,16.85,1,1,96.4,0.0,1,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0,3,13.97,0,1,59.5,0.0,-1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,0,11,16.33,1,1,62.1,0.0,-1,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
sample_valid_data_x=sample_validation_data.drop(target,axis=1)
print(model1.predict(sample_valid_data_x))

print(sample_validation_data[target])

[ 1  1 -1  1]
19    1
79    1
24   -1
41   -1
Name: safe_loans, dtype: int64


**Quiz Question**: What percentage of the predictions on sample_validation_data did decision_tree_model get correct?

75%

In [89]:
print(model1.predict_proba(sample_valid_data_x))

[[0.4635469  0.5364531 ]
 [0.4635469  0.5364531 ]
 [0.58922559 0.41077441]
 [0.39804951 0.60195049]]


**Quiz Question**: Which loan has the highest probability of being classified as a safe loan?

In [90]:
print(model2.predict_proba(sample_valid_data_x))

[[0.42788678 0.57211322]
 [0.42788678 0.57211322]
 [0.6356058  0.3643942 ]
 [0.42788678 0.57211322]]


In [91]:
print(model2.predict(sample_valid_data_x))

[ 1  1 -1  1]


In [95]:
from sklearn.metrics import accuracy_score
accuracy_score(valid[target],y1)

0.6097587246876346

In [96]:
accuracy_score(valid[target], y2)

0.5996337785437311

**Quiz Question**: What is the accuracy of decision_tree_model on the validation set, rounded to the nearest .01?

In [104]:
model3=tree.DecisionTreeClassifier(max_depth=10)
model3.fit(x,y)
y3_t=model3.predict(x)

In [107]:
print(accuracy_score(train[target],y3_t))

0.649097356544165


In [109]:
print(accuracy_score(valid[target],model3.predict(x_valid)))

0.6060965101249461


In [114]:
print(accuracy_score(valid[target],model1.predict(x_valid)))

0.6097587246876346


**Quiz Question**:How does the performance of big_model on the validation set compare to decision_tree_model on the validation set? Is this a sign of overfitting?

In [115]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(valid[target],model1.predict(x_valid)).ravel()
print(fp)
print(fn)

2226
1397


**Quiz Question**: Let's assume that each mistake costs us money: a false negative costs $ 10000 while a false positive positive costs $20000. What is the total cost of mistakes made by decision_tree_model on validation_data?

In [116]:
print(10000*fn+20000*fp)

58490000
